In [ ]:
pip install pandas psycopg2

In [ ]:
import pandas as pd
import psycopg2
import os

conn = psycopg2.connect(
    dbname="f1",
    user="postgres",
    password="Theshield193",
    host="localhost"
)

sql = """
WITH weather_avg as (
    SELECT session_key, AVG(track_temperature) as avg_track_temperature
    FROM weather
    GROUP BY session_key
)
SELECT 
    l.lap_number,
    l.duration_sector_1,
    l.duration_sector_2,
    l.duration_sector_3,
    s.session_type,
    st.compound,
    st.tyre_age_at_start,
    w.avg_track_temperature as track_temperature,
    d.driver_number,
    l.session_key
FROM laps l
JOIN sessions s ON l.session_key = s.session_key
JOIN stint st ON l.session_key = st.session_key AND l.driver_number = st.driver_number and l.lap_number between st.lap_start and st.lap_end
JOIN weather_avg w ON l.session_key = w.session_key
JOIN driver d on l.driver_number = d.driver_number
WHERE s.session_type = 'Race'
ORDER BY l.lap_number, l.session_key
;
"""

# Load the sql data in a pandas dataframe
df = pd.read_sql(sql,conn)

df['compound'] = df['compound'].astype(str);
df['session_type'] = df['session_type'].astype(str);

df.head()

# df.groupby(['session_key', 'lap_number', 'driver_number']).size()

output_folder = "../data"

csv_path = os.path.join(output_folder, "f1_lap_training_csv")
print(f"Path: {csv_path}")
df.to_csv(csv_path, index=False)

print(f"✅ Dataset created: {csv_path}")